In [1]:
import os

path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

In [7]:
path_parent

'/Users/gieunkwak/Data Analytics/Repositories/platform'

In [23]:
import pandas as pd
from fredapi import Fred
import json
from pathlib import Path
import investpy
import requests
from urllib.request import urlopen
import yfinance as yf
from datetime import datetime, date
# from log import get_logger

# auth
# basepath = Path(__file__).parent.parent

# logger = get_logger(__name__)

with open('app/keys/keys.json', 'r') as key_file:
    keys = json.load(key_file)

fred = Fred(api_key=keys['fred'])
alpha_vantage = keys['alpha_vantage']
financial_modeling_prep = keys['financial_modeling_prep']

In [3]:
financial_modeling_prep

'42573d51ce11e37bf7503838f5c3f6ec'

In [33]:
class FMP:
    """
    Class object for FinancialModelingPrep
    """

    def __init__(self):
        self.key = financial_modeling_prep
#         logger.info("Financial Modeling Prep api ready.")

    def get_jsonparsed_data(self, url):
        """
        Receive the content of ``url``, parse it as JSON and return the object.

        Args:
            url (str): url of the API

        Returns:
            parsed_content (json): parsed content returned from the API
        """
        response = urlopen(url)
        data = response.read().decode("utf-8")
        return json.loads(data)

    def get_historical_daily_price(self, call_type, symbol):
        """
        Retrieve historical price data. daily.
        
        
        
        """
#         logger.info("Fetching daily price data of {}".format(symbol))

        url = "https://financialmodelingprep.com/api/v3/historical-price-full/{}?apikey={}".format(symbol, self.key)
        daily_price = self.get_jsonparsed_data(url)

        df = pd.DataFrame(daily_price['historical'])
        df['symbol'] = daily_price['symbol']
        df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

        return df
    
    def get_economic_index(self, symbol, start_date, end_date):
        """
        Retrieve economic index
        
        Total list of available indices: updated 2022-10-01
        
        "GDP", "realGDP", "nominalPotentialGDP", "realGDPPerCapita",
        "federalFunds", "CPI", "inflationRate", "inflation", "retailSales", "consumerSentiment", "durableGoods",
        "unemploymentRate", "totalNonfarmPayroll", "initialClaims", "industrialProductionTotalIndex",
        "newPrivatelyOwnedHousingUnitsStartedTotalUnits", "totalVehicleSales", "retailMoneyFunds",
        "smoothedUSRecessionProbabilities", "3MonthOr90DayRatesAndYieldsCertificatesOfDeposit",
        "commercialBankInterestRateOnCreditCardPlansAllAccounts", "30YearFixedRateMortgageAverage",
        "15YearFixedRateMortgageAverage"
        """
        
        url = "https://financialmodelingprep.com/api/v4/economic?name={}&from={}&to={}&apikey={}".format(symbol, start_date, end_date, self.key)
        historical_index = self.get_jsonparsed_data(url)
        
        df = pd.DataFrame(historical_index)
        df.columns = ['date', symbol]
        df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
        
        return df

In [34]:
fmp = FMP()

In [39]:
gdp = fmp.get_economic_index("inflationRate", start_date, today)

In [40]:
gdp

,date,inflationRate
0,2022-09-29,2.19
1,2022-09-28,2.33
2,2022-09-27,2.33
3,2022-09-26,2.32
4,2022-09-23,2.37
...,...,...
4937,2003-01-08,1.71
4938,2003-01-07,1.62
4939,2003-01-06,1.63
4940,2003-01-03,1.62


In [ ]:
daily_sp500

In [41]:
url = "https://financialmodelingprep.com/api/v3/historical-price-full/{}?from={}&to={}&apikey={}".format("AAPL", start_date, today, fmp.key)
daily_price = get_jsonparsed_data(url)

In [43]:
df = pd.DataFrame(daily_price['historical'])
df['symbol'] = daily_price['symbol']
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [44]:
df

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,symbol
0,2022-09-30,141.279999,143.100006,138.000000,138.199997,138.199997,103261884.0,103261884.0,-3.08000,-2.180,139.76667,"September 30, 22",-0.02180,AAPL
1,2022-09-29,146.100006,146.720001,140.679993,142.479996,142.479996,128138200.0,128138200.0,-3.62001,-2.478,143.29333,"September 29, 22",-0.02478,AAPL
2,2022-09-28,147.639999,150.639999,144.839996,149.839996,149.839996,146691400.0,146691400.0,2.20000,1.490,148.44000,"September 28, 22",0.01490,AAPL
3,2022-09-27,152.740005,154.720001,149.949997,151.759995,151.759995,84442700.0,84442700.0,-0.98001,-0.642,152.14333,"September 27, 22",-0.00642,AAPL
4,2022-09-26,149.660004,153.770004,149.639999,150.770004,150.770004,93339400.0,93339400.0,1.11000,0.742,151.39334,"September 26, 22",0.00742,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5719,2000-01-07,0.861607,0.901786,0.852679,0.888393,0.758538,460734400.0,460734400.0,-0.10307,-11.963,0.83767,"January 07, 00",-0.11963,AAPL
5720,2000-01-06,0.947545,0.955357,0.848214,0.848214,0.724232,767972800.0,767972800.0,-0.22331,-23.567,0.84260,"January 06, 00",-0.23567,AAPL
5721,2000-01-05,0.926339,0.987165,0.919643,0.928571,0.792844,778321600.0,778321600.0,-0.13350,-14.412,0.89988,"January 05, 00",-0.14412,AAPL
5722,2000-01-04,0.966518,0.987723,0.903460,0.915179,0.781409,512377600.0,512377600.0,-0.18511,-19.152,0.89086,"January 04, 00",-0.19152,AAPL


In [24]:
start_date = '2000-01-01'
today = date.today().strftime("%Y-%m-%d")

In [26]:
symbol = 'realGDP'
url = "https://financialmodelingprep.com/api/v4/economic?name={}&from={}&to={}&apikey={}".format(symbol, start_date, today, fmp.key)
historical_index = get_jsonparsed_data(url)

In [ ]:
df = pd.DataFrame(historical_index)
df.columns = ['date', symbol]

df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df['value'] = symbol


In [27]:
historical_index

[{'date': '2022-04-01', 'value': 19895.271},
 {'date': '2022-01-01', 'value': 19924.088},
 {'date': '2021-10-01', 'value': 20006.181},
 {'date': '2021-07-01', 'value': 19672.594},
 {'date': '2021-04-01', 'value': 19544.248},
 {'date': '2021-01-01', 'value': 19216.224},
 {'date': '2020-10-01', 'value': 18924.262},
 {'date': '2020-07-01', 'value': 18743.72},
 {'date': '2020-04-01', 'value': 17378.712},
 {'date': '2020-01-01', 'value': 18989.877},
 {'date': '2019-10-01', 'value': 19215.691},
 {'date': '2019-07-01', 'value': 19130.932},
 {'date': '2019-04-01', 'value': 18962.175},
 {'date': '2019-01-01', 'value': 18835.411},
 {'date': '2018-10-01', 'value': 18733.741},
 {'date': '2018-07-01', 'value': 18699.748},
 {'date': '2018-04-01', 'value': 18565.697},
 {'date': '2018-01-01', 'value': 18437.127},
 {'date': '2017-10-01', 'value': 18310.3},
 {'date': '2017-07-01', 'value': 18127.994},
 {'date': '2017-04-01', 'value': 17979.218},
 {'date': '2017-01-01', 'value': 17889.094},
 {'date': '20

In [ ]:
"GDP", "realGDP", "nominalPotentialGDP", "realGDPPerCapita",
"federalFunds", "CPI", "inflationRate", "inflation", "retailSales", "consumerSentiment", "durableGoods",
"unemploymentRate", "totalNonfarmPayroll", "initialClaims", "industrialProductionTotalIndex",
"newPrivatelyOwnedHousingUnitsStartedTotalUnits", "totalVehicleSales", "retailMoneyFunds",
"smoothedUSRecessionProbabilities", "3MonthOr90DayRatesAndYieldsCertificatesOfDeposit",
"commercialBankInterestRateOnCreditCardPlansAllAccounts", "30YearFixedRateMortgageAverage",
"15YearFixedRateMortgageAverage"